##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 기초: 첫번째 신경망(Neural Network) 모델(model) 훈련

여기서 첫번째 머신러닝(Machine Learning) 신경망(Neural Network) 모델(model)을 학습하게 됩니다!

이번 실습에서는 간단하고 기본적인 개념(basic concept)만을 익혀 보도록 합니다. 나중에 Colabs는보다 발전된(advanced) 문제를 다룹니다.

우리가 해결할 문제는 섭씨에서 화씨로 변환하는 것입니다. 대략적인 공식은 다음과 같습니다:

$$ f = c \times 1.8 + 32 $$


앞에서 다루었 듯이 이 공식을 일반적인 파이썬 함수(function)으로 만드는 것은 간단하지만 기계 학습은 아닙니다.

TensorFlow에 몇 가지 샘플 섭씨 값 (0, 8, 15, 22, 38)과 해당 화씨 값 (32, 46, 59, 72, 100)을 만들고, 위의 공식을 훈련(train) 과정을 통해 파악하는 모델(model)을 만들어 볼것 있습니다.

## 디펜던시(dependencies) 라이브러리 불러오기(import)

먼저 TensorFlow를 가져옵니다. 여기서는 사용의 편의를 위해 'tf'라고 부릅니다. 

TensorFlow 로그는 오류만 표시하도록 알려줍니다.

다음으로, [넘피(Numpy)](http://www.numpy.org/)를 'np'로 가져 오십시오. 넘피(Numpy)는 우리가 데이터를 성능이 높은(highly performant) 목록으로 표현할 수 있도록 도와줍니다.


In [ ]:
import tensorflow as tf
import numpy as np

## 학습/트레이닝 데이터 셋업(Set up training data)

전에 보았듯이, 머신러닝의 지도학습은 입력과 출력의 집합이 주어진 알고리즘을 알아내는 것입니다.

여기에서 섭씨로 온도가 주어질 때 화씨로 온도를 줄 수있는 모델을 만듭니다.

모델을 훈련시키는 데 사용할 수 있는`celsius_q`와`fahrenheit_a`라는 두 개의 리스트(혹은 배열 array)를 만듭니다.


In [ ]:
celsius_q    = np.array([-40, -10, 0, 8, 15, 22, 38], dtype=float )
fahrenheit_a = np.array([-40, 14, 32, 46, 59, 72, 100], dtype=float )

# 만든 2개의 배열을 돌려서 값을 프린트로 확인
for i, c in enumerate(celsius_q):
  print("{} 섭씨는 = {} 화씨".format(c, fahrenheit_a[i]))

### 머신러닝(Machine Learning) 용어

 - **피쳐(Feature)** — 모델의 입력입니다. 이 경우 단일 값 - 섭씨(celsius)

 - **레이블(Labels)** — 모델에서 예측 한 결과입니다. 이 경우 단일 값 - 화씨(fahrenheit)
 
 - **예제(Example)** — 교육 중에 사용되는 입력/출력 페어(pair). 여기서는 `(22,72)`와 같은 특정 인덱스(index)에서 `celsius_q` 및 `fahrenheit_a`의 값 pair을 사용합니다.

## 모델 만들기

이제 모델을 만듭니다. 가능한 가장 단순한 모델인 Dense 네트워크(network)를 사용할 것입니다. 이 화씨섭씨 문제는 간단하기 때문에 이 네트워크에는 단일 뉴런(neuron)이 있는 단일 레이어(layer)만 사용할 것 입니다.

### 레이어(layer) 만들기

레이어 `l0`(엘영) 이란 변수를 만들고 다음과 같은 설정으로`tf.keras.layers.Dense`를 인스턴스화 합니다:

* `input_shape = [1]` - 이 레이어에 대한 입력이 단일 값임을 지정합니다. 즉, 모양은 멤버가 하나인 1 차원 배열입니다. 이것이 첫번째 (그리고 유일한) 레이어이기 때문에, 그 입력 모양은 전체 모델의 입력 모양입니다. 단일 값은 섭씨 온도를 나타내는 부동소수점(float point) 숫자입니다.

* `units = 1` - 레이어의 뉴런(neuron) 수를 지정합니다. 뉴런 수는 얼마나 많은 내부 변수(internal variables)가 문제를 해결할 방법을 배우고자 하는지를 정의합니다. 또한, 이 경우는 레이어가 하나이기 때문에 모델의 출력 크기(size)이며 화씨 온도를 나타내는 부동소수점(float) 값이기도 합니다. (다중 레이어(multi-layered) 네트워크에서 레이어의 크기와 모양은 다음 레이어의 'input_shape'와 일치해야합니다.)


In [ ]:
# l0(엘0)이란 변수 이름으로 레이어 생성
l0 = tf.keras.layers.Dense(units = 1, input_shape=[1]) 

# l0을 사용해서 모델 만들기
model = tf.keras.Sequential([l0])

### 레이어를 가지고 모델 만들기

레이어가 정의되면 모델을 레이어를 이용해 조립(assemble)합니다. Sequential 모델 정의는 레이어 리스트(list)을 인수(argument)로 사용하고, 입력에서 출력까지의 계산 순서를 지정합니다.


이 모델은 레이어, l0(엘영) 하나만 갖습니다.

In [ ]:
# 모델 컴파일
model.compile(loss='mean_squared_error',
             optimizer=tf.keras.optimizers.Adam(0.1))

**노트**

이 두가지를 동시에 할 수도 있습니다.

아래와 같이 모델을 만들때, 동시에 레이어를 정의하는 코드를 흔히 볼 수 있습니다 :


```python
model = tf.keras.Sequential([
  tf.keras.layers.Dense(units=1, input_shape=[1])
])
```

## 손실(loss) 및 최적화(optimizer) 함수(functions) 사용하여 모델 컴파일하기

학습(training) 전에 모델을 컴파일 해야 합니다. 
컴파일때 필요한 두가지는 아래와 같습니다:

- **손실 함수(Loss function)** — 예측 결과가 원하는 결과로부터 얼마나 떨어져 있는지 측정하는 방법. (측정 된 차이를 "손실"이라고합니다.)

- **최적화 함수(Optimizer function)** — 손실을 줄이기 위해 내부 값을 조정하는 방법입니다.


In [ ]:
# 모델 컴파일
model.compile(loss='mean_squared_error',
             optimizer=tf.keras.optimizers.Adam(0.1))

이 두개는 학습(training)중 사용되며 (다음에 나오는 model.fit()) 각 포인트(each point)에서 손실을 먼저 계산 한 다음 그것을 향상(improve) 시킵니다. 실제로 정확하게 이야기하면, 학습(training)이란 현재 손실을 계산하고 개선(improve)하는 것이라 할수 있습니다.

Training중, 옵티마이저 함수(optimizer function)는 모델의 내부변수(internal variables)를 조정/ 계산하는 데 사용됩니다. 여기서 목표는 모델 (정확하게는 그냥 수학공식)이 섭씨를 화씨로 변환하기 위한 실제 공식과 가까워 질때까지 내부 변수를 조정하는 것입니다.

TensorFlow는이 튜닝(tuning)을 수행(perform)하기 위해 수치해석(numerical analysis)을 사용합니다. 이때 숨겨진 복잡한 일이 일어나는데  그 부분은 나중에 자세히 알아보도록 하겠습니다. 지금 이 매개 변수(parameters) 에 대해 알아야 할 것은 다음과 같습니다:

여기에 사용 된 손실함수(loss function) - 평균 제곱 오차([mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error))와 옵티마이저(optimizer) - 아담(Adam)은 단순한 모델에 주로 사용되고 있지만 많은 다른 모델도 사용할 수 있습니다. 이러한 특정 기능이 어디에 어떻게 작동 하는지는 나중에 보도록 하겠습니다.

하나 알아두면 좋은것은, 위의 코드에서 옵티마이저가 사용하는 0.1은 학습 속도라는 것입니다. 이 값은 모델에서 값을 조정할 때 사용되는 단계 크기(step size)입니다. 값이 너무 작으면 모델을 학습하는데 너무 많은 반복이 필요하고, 너무 크면 정확도가 떨어집니다. 정확한 값을 찾으려면 다양한 값을 넣어서 계속 돌려봐야 하며, 보통 범위는 0.001(default)에서 0.1사이 입니다.

## 모델 학습(train)하기

`fit` 메소드를 이용해서 모델을 training 합니다.

training중 모델은 섭씨 값을 받고, 현재 내부변수(internal variable) - 가중치(weight)를 사용하여 계산하고 화씨에 해당하는 값을 출력(output)합니다. 
가중치(weight)는 초기에 무작위로 설정됩니다. 따라서, 출력은 정확한 값에 근접하지 않습니다. 실제 출력(actual weight)과 원하는 출력(desired weight)간의 차이는 손실함수(loss function)를 사용하여 계산되며 옵티마이저함수(optimizer function)은 가중치를 어떻게 조정할지 결정합니다.

계산, 비교, 조정하는 사이클(cycle)은 `fit` 메소드가 콘트롤합니다. 첫 번째 인수는 입력(inputs)이고, 두 번째 인수는 원하는 출력(desired output)입니다. `epochs` 인수는 사이클를 몇번 실행해야하는지 지정하고 verbose 인수는 메소드가 생성하는 출력의 양을 정합니다.


In [ ]:
history = model.fit(celsius_q, fahrenheit_a,
                    epochs = 500,
                    verbose = True)
print("Finished training the model")

다음 모듈에서 더 자세한 내용과 Dense 레이어가 실제로 내부적으로 어떻게 작동하는지에 대해 알아 보기로 합니다 .

## training 통계 보기

`fit` 메소드는 history 객체(object)를 반환(return)합니다. 이 object를 사용하여 각 training epoch 후에 모델의 손실이 어떻게 감소하는지 플랏(plot) 할 수 있습니다. 손실이 많다는 것은 모델이 예측하는 화씨 온도가 `fahrenheit_a`의 해당 값과 멀리 떨어져 있음을 의미합니다.

여기서는 [Matplotlib](https://matplotlib.org/)을 사용하여 시각화 합니다 (다른 도구를 사용할 수도 있음). 보시다시피, 이 모델은 처음에는 매우 빨리 향상되고, 끝부분에는 "완벽(perfect)"해 질때까지 꾸준이(steady) 천천히 값을 맞추어 나갑니다.

In [ ]:
import matplotlib.pyplot as plt
# x label, y label을 설정하고 loss를 설정후 plot()을 콜 합니다.
plt.xlabel('Epoch Number')
plt.ylabel("Loss Magnitude")
plt.plot(history.history['loss'])

## 모델을 사용하여 값 예측하기

이제`celsius_q`와`fahrenheit_a` 사이의 관계를 배우도록 훈련 된 모델이 있습니다. `predict()` 메소드를 사용하여 예측하면 이전에 몰랐던 섭씨도에 대한 화씨도를 계산할 수 있습니다.

예를 들어 섭씨 값이 100 인 경우 화씨 결과는 어떻게 될까요? 이 코드를 실행하기 전에 추측해 보십시오.


In [ ]:
# 모델을 사용하여 값 100을 예측하고 프린트
print(model.predict([100.0]))


정답은  $100 \times 1.8 + 32 = 212$ 입니다. 우리가 만들 모델이 잘 작동하는걸 확인할 수 있습니다.

### 검토(Review)

* 우리는 Dense 레이어가 있는 모델을 만들었습니다.
* 우리는 3500가지의 예(7 pairs, 500번 넘게 반복으로 돌림)로 학습 했습니다.

이 모델은 섭씨 값에 대해 정확한 화씨 값을 출력 할 수 있을 때까지 dense layer의 변수(가중치)를 조정(tuning)했습니다. (섭씨 100은 우리의 training 데이터의 일부가 아니었습니다.)



## 레이어(layer) 가중치(weight) 살펴보기

마지막으로 Dense layer의 내부변수(internal variable)를 print 해 봅시다.


In [ ]:
# 내부변수 - 가중치 값 프린트
print("These are the layer variables: {}".format(l0.get_weights()))

첫 번째 변수는 ~ 1.8에 가까우며 두 번째 변수는 ~ 32에 가깝습니다. 이 값 (1.8 및 32)은 실제 변환 공식의 실제 변수입니다.

이 값은 변환 공식의 값에 아주 가깝습니다. 다음 모듈에서 Dense layer가 어떻게 작동하는지 설명하겠지만, 하나의 입력과 하나의 출력을 가진 단 하나의 뉴런의 경우, 내부(internal) 수학공식은  [the equation for a line](https://en.wikipedia.org/wiki/Linear_equation#Slope%E2%80%93intercept_form) $y = mx + b$에 대한 방정식과 동일하게 보입니다. 마치 변환 공식 $f = 1.8c + 32$와 같다는걸 알 수 있습니다.

형식(form)이 같기 때문에 변수는 1.8과 32의 표준(standard)값을 커버할수(coverage) 있습니다. 이것이 안에서 정확히 무슨 일이 일어 났는지 알수 있는 증거가 됩니다. 

추가 뉴런, 추가 입력 및 추가 출력이 생기면 수식은 훨씬 더 복잡해 지지만 아이디어는 같습니다.




## 작은실험 - 레이어 더하기

단위가 다른 여러개의 Dense layer를 만들어 더하면 변수는 어떻게 달라지고 계산은 어떻게 달라지는 실험해 보도록 해보겠습니다.

In [ ]:
l0 = tf.keras.layers.Dense(units=4, input_shape=[1])  
l1 = tf.keras.layers.Dense(units=4)  
l2 = tf.keras.layers.Dense(units=1)  
model = tf.keras.Sequential([l0, l1, l2])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1))
model.fit(celsius_q, fahrenheit_a, epochs=500, verbose=False)
print("Finished training the model")
print(model.predict([100.0]))
print("Model predicts that 100 degrees Celsius is: {} degrees Fahrenheit".format(model.predict([100.0])))
print("These are the l0 variables: {}".format(l0.get_weights()))
print("These are the l1 variables: {}".format(l1.get_weights()))
print("These are the l2 variables: {}".format(l2.get_weights()))

보시다시피 이 모델은 해당 화씨 값을 실제로 잘 예측할 수 있습니다. 그러나 `l0` 과 `l0` 레이어의 변수 (가중치)를 보면 ~ 1.8과 32에 가까운 것조차 없습니다. 추가 된 layer들은 변환 방정식을 복잡하게 만들면서 "단순한" 형태로 알아볼 수 없게 됩니다.

다음 모듈에서 Dense layer가 작동하는 방법에 대해 좀 더 자세히 알아보록 합니다.
